In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 458 kB/s 
     |████████████████████████████████| 7.6 MB 65.2 MB/s 
     |████████████████████████████████| 182 kB 67.1 MB/s 


In [ ]:
import cv2 as cv
import os
from transformers import ViTFeatureExtractor, ViTModel
from skimage import io
from skimage.transform import resize
import numpy as np
import torch


def keyframe_sort(element):
  element = element.split("_")
  return int(element[1])

def tokenize_img_function(image):
  feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
  model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
  inputs = feature_extractor(image, return_tensors="pt")
  with torch.no_grad():
    outputs = model(**inputs)
  last_hidden_states = outputs.last_hidden_state
  return last_hidden_states

def extract_image_features(image_name):
  image_name = path + "/"+image_name
  image = io.imread(image_name)
  image = resize(image, (224, 224))
  result = tokenize_img_function(image)
  return result

directories_pathnames = ['accident_Cropped', 'call_Cropped',
                         'doctor_Cropped', 'help_Cropped',
                         'hot_Cropped', 'lose_Cropped',
                         'pain_Cropped', 'thief_Cropped','accident_Raw', 'call_Raw',
                         'doctor_Raw', 'help_Raw',
                         'hot_Raw', 'lose_Raw',
                         'pain_Raw', 'thief_Raw']

directories_pathnames1 = ['accident_Raw', 'call_Raw',
                         'doctor_Raw', 'help_Raw',
                         'hot_Raw', 'lose_Raw',
                         'pain_Raw', 'thief_Raw']

label_dict = {
    'accident_Cropped': 0,
    'accident_Raw': 0,
    'call_Cropped': 1,
    'call_Raw': 1,
    'doctor_Cropped': 2,
    'doctor_Raw': 2,
    'help_Cropped': 3,
    'help_Raw': 3,
    'hot_Cropped': 4,
    'hot_Raw': 4,
    'lose_Cropped': 5,
    'lose_Raw': 5,
    'pain_Cropped': 6,
    'pain_Raw': 6,
    'thief_Cropped': 7,
    'thief_Raw': 7,
}

all_features = []
all_label = []
prev_len = 0


# dir_list = os.listdir(path)

for direc in directories_pathnames:
  print('creating feature and labels running for: ', direc)
  path = "/content/drive/MyDrive/NNDL_Project/"
  path = path + direc
  dir_list = os.listdir(path)
  dir_list.sort(key=keyframe_sort)
  count = 1
  for i in range(0, len(dir_list), 3):
    feature_1 = np.array(extract_image_features(dir_list[i]))
    feature_2 = np.array(extract_image_features(dir_list[i+1]))
    feature_3 = np.array(extract_image_features(dir_list[i+2]))

    feature_concat = np.concatenate((feature_1, feature_2, feature_3), axis=None)
    all_features.append(feature_concat)
    print('features created: ' + str(count))
    count += 1

  dif = len(all_features) - prev_len
  all_label.extend([label_dict[direc]]*dif)
  prev_len += dif



creating feature and labels running for:  accident_Cropped


Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/346M [00:00<?, ?B/s]

features created: 1
features created: 2
features created: 3
features created: 4
features created: 5
features created: 6
features created: 7
features created: 8
features created: 9
features created: 10
features created: 11
features created: 12
features created: 13
features created: 14
features created: 15
features created: 16
features created: 17
features created: 18
features created: 19
features created: 20
features created: 21
features created: 22
features created: 23
features created: 24
features created: 25
features created: 26
features created: 27
features created: 28
features created: 29
features created: 30
features created: 31
features created: 32
features created: 33
features created: 34
features created: 35
features created: 36
features created: 37
features created: 38
features created: 39
features created: 40
features created: 41
features created: 42
features created: 43
features created: 44
features created: 45
features created: 46
features created: 47
features created: 48
f

In [ ]:
import pandas as pd
print(pd.__version__)

print(len(all_features))
print(all_features[1].shape)


1.3.5
824
(453888,)


In [ ]:
# Saving features and corresponding labels

import pandas as pd
import numpy as np

df = pd.DataFrame({"features": all_features, "labels": all_label})
# df.to_csv("featureExtraction.csv", index=False)
df.to_pickle("featureExtraction_combined.pkl")
